In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps
import os

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")

    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})

    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

class Config():
    training_dir = "/content/drive/MyDrive/siamese-net/content/sign_data/train"
    testing_dir = "/content/drive/MyDrive/siamese-net/content/sign_data/test"
    train_batch_size = 64
    train_number_epochs = 100

In [ ]:
class SiameseNetwork(tf.keras.Model):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        
        self.cnn1 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(4, kernel_size=3, padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(8, kernel_size=3, padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(8, kernel_size=3, padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization()
        ])
        
        self.fc1 = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(500, activation='relu'),
            tf.keras.layers.Dense(500, activation='relu'),
            tf.keras.layers.Dense(5)
        ])

    def call(self, x):
        output = self.cnn1(x)
        output = self.fc1(output)
        return output


In [ ]:
class SiameseNetworkDataset(tf.keras.utils.Sequence):
    def __init__(self, imageFolderDataset, transform=None, should_invert=True):
        self.imageFolderDataset = imageFolderDataset
        self.transform = transform
        self.should_invert = should_invert

    def __getitem__(self, index):
        img0_tuple = random.choice(self.imageFolderDataset.filepaths)
        should_get_same_class = random.randint(0, 1)

        if should_get_same_class:
            while True:
                img1_tuple = random.choice(self.imageFolderDataset.filepaths)
                if self.imageFolderDataset.directory == os.path.dirname(img1_tuple):
                    break
        else:
            while True:
                img1_tuple = random.choice(self.imageFolderDataset.filepaths)
                if self.imageFolderDataset.directory != os.path.dirname(img1_tuple):
                    break

        img0 = Image.open(img0_tuple)
        img1 = Image.open(img1_tuple)
        img0 = img0.convert("L")
        img1 = img1.convert("L")

        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)

        img0 = np.array(img0)
        img1 = np.array(img1)

        img0 = np.expand_dims(img0, axis=-1)
        img1 = np.expand_dims(img1, axis=-1)

        img0 = tf.image.resize(img0, (100, 100))
        img1 = tf.image.resize(img1, (100, 100))

        label = int(self.imageFolderDataset.directory != os.path.dirname(img1_tuple))
        label = np.array([label], dtype=np.float32)

        return (img0, img1), label

    def __len__(self):
        return len(self.imageFolderDataset.filepaths)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_datagen = ImageDataGenerator(rescale=1.0/255.0)

folder_dataset = image_datagen.flow_from_directory(
    directory=Config.training_dir,
    target_size=(100, 100),
    batch_size=1,
    class_mode=None,
    shuffle=True
)

siamese_dataset = SiameseNetworkDataset(
    imageFolderDataset=folder_dataset,
    transform=None,
    should_invert=False
)

In [ ]:
vis_dataloader = tf.data.Dataset.from_generator(
    lambda: siamese_dataset,
    output_signature=(
        (tf.TensorSpec(shape=(100, 100, 1), dtype=tf.float32),
         tf.TensorSpec(shape=(100, 100, 1), dtype=tf.float32)),
        tf.TensorSpec(shape=(2,), dtype=tf.float32)
    )
).shuffle(buffer_size=len(siamese_dataset)).batch(8).prefetch(buffer_size=tf.data.AUTOTUNE)
print(vis_dataloader)
dataiter = iter(vis_dataloader)
print(dataiter)
example_batch = next(dataiter)
print(example_batch)

concatenated = tf.concat([example_batch[0][0], example_batch[0][1]], axis=0)

grid = tf.squeeze(tf.concat(tf.unstack(concatenated), axis=-2))
plt.imshow(grid, cmap='gray')
plt.axis('off')
plt.show()

print(example_batch[1].numpy())

In [ ]:
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, margin=2.0, **kwargs):
        super(ContrastiveLoss, self).__init__(**kwargs)
        self.margin = margin

    def call(self, output1, output2, label):
        euclidean_distance = tf.norm(output1 - output2, axis=-1, keepdims=True)
        loss_contrastive = tf.reduce_mean((1 - label) * tf.square(euclidean_distance) +
                                          label * tf.square(tf.maximum(0.0, self.margin - euclidean_distance)))

        return loss_contrastive

In [ ]:
import tensorflow as tf

train_dataloader = tf.data.Dataset.from_generator(
    lambda: siamese_dataset,
    output_signature=(
        (tf.TensorSpec(shape=(100, 100, 1), dtype=tf.float32),
         tf.TensorSpec(shape=(100, 100, 1), dtype=tf.float32)),
        tf.TensorSpec(shape=(1,), dtype=tf.float32)
    )
).shuffle(buffer_size=len(siamese_dataset)).batch(Config.train_batch_size).prefetch(tf.data.AUTOTUNE)

net = SiameseNetwork()
criterion = ContrastiveLoss()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

counter = []
loss_history = []
iteration_number = 0

@tf.function
def train_step(img0, img1, label):
    with tf.GradientTape() as tape:
        output1, output2 = net(img0, img1)
        loss_contrastive = criterion(output1, output2, label)
    gradients = tape.gradient(loss_contrastive, net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, net.trainable_variables))
    return loss_contrastive

for epoch in range(Config.train_number_epochs):
    for i, data in enumerate(train_dataloader):
        img0, img1, label = data
        loss = train_step(img0, img1, label)

        if i % 10 == 0:
            print("Epoch number {}\n Current loss {}\n".format(epoch, loss.numpy()))
            iteration_number += 10
            counter.append(iteration_number)
            loss_history.append(loss.numpy())

net.save_weights("siamese_model.h5")
show_plot(counter, loss_history)


# folder_dataset_test = dset.ImageFolder(root=Config.testing_dir)
folder_dataset_test = dset.ImageFolder(root="/content/drive/MyDrive/siamese-net/content/sign_data/jeong")

siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor()
                                                                      ])
                                       ,should_invert=False)

 

test_dataloader = DataLoader(siamese_dataset,num_workers=6,batch_size=1,shuffle=True)

dataiter = iter(test_dataloader)
x0,_,_ = next(dataiter)

for i in range(10):
    _,x1,label2 = next(dataiter)
    concatenated = torch.cat((x0,x1),0)

    output1,output2 = net(Variable(x0).cuda(),Variable(x1).cuda())
    euclidean_distance = F.pairwise_distance(output1, output2)

    imshow(torchvision.utils.make_grid(concatenated),'Dissimilarity: {:.2f}'.format(euclidean_distance.item()))

In [ ]:
import tensorflow_datasets as tfds

# Load dataset
dataset, info = tfds.load('image_folder', split='test', data_dir=Config.testing_dir, with_info=True)

# Define image preprocessing function
def preprocess_image(image):
    image = tf.image.resize(image, (100, 100))
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

# Create SiameseNetworkDataset class
class SiameseNetworkDataset(tf.keras.utils.Sequence):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __getitem__(self, index):
        sample = self.dataset[index]
        img0 = preprocess_image(sample['image'])
        label0 = sample['label']
        
        should_get_same_class = np.random.randint(0, 2)
        
        if should_get_same_class:
            while True:
                sample_same = self.dataset[np.random.randint(len(self.dataset))]
                if sample_same['label'] == label0:
                    break
        else:
            while True:
                sample_diff = self.dataset[np.random.randint(len(self.dataset))]
                if sample_diff['label'] != label0:
                    break
        
        img1 = preprocess_image(sample_same['image'])
        label1 = sample_same['label']
        
        img0 = tf.expand_dims(img0, axis=-1)
        img1 = tf.expand_dims(img1, axis=-1)
        
        return img0, img1, label0, label1

    def __len__(self):
        return len(self.dataset)

# Create SiameseNetwork model
def SiameseNetwork():
    # Define your model architecture here
    pass

# Load trained model
model = SiameseNetwork()

siamese_dataset = SiameseNetworkDataset(dataset, transform=preprocess_image)

test_dataloader = tf.data.Dataset.from_generator(
    lambda: siamese_dataset,
    output_signature=(
        tf.TensorSpec(shape=(100, 100, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(100, 100, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int64),
        tf.TensorSpec(shape=(), dtype=tf.int64)
    )
).batch(1)

dataiter = iter(test_dataloader)
x0, _, label0, _ = next(dataiter)

for i in range(10):
    _, x1, _, label1 = next(dataiter)
    concatenated = tf.concat([x0, x1], axis=0)

    output1, output2 = net(x0, x1)
    euclidean_distance = tf.norm(output1 - output2, ord='euclidean')

    plt.imshow(np.squeeze(concatenated), cmap='gray')
    plt.title('Dissimilarity: {:.2f}'.format(euclidean_distance.numpy()))
    plt.axis('off')
    plt.show()
